# First order differential equations

In the following, we solve the following 1st order inhomogenous differential equation using Python's scipy Library. Let $A\in \mathbb{C}^{n \times n}$ with $n\in \mathbb{N}$

$\begin{align}
\frac{\partial^{n} }{\partial t^{n} }x(t) = A \, x(t) + u(t)\end{align}$

## Homogenous part

The following equation denotes the homogenous part of the equation:

$\begin{align}
\frac{\partial^{n} }{\partial t^{n} }x(t) = A \, x(t) \\

\end{align}$ 

And the following code solves it:


In [1]:
# HOMOGENnous Solution

import numpy as np
from scipy.linalg import eig
import sympy as sp
a = np.array([1, 2, 3])
print('Numpy array:', a)

Numpy array: [1 2 3]\n

## Inhomogenous part
This part solves the inhomogenous part of the equation.

which was to be shown.